In [ ]:
from importlib import reload
import models.networks as nm
import models.ganomaly as gm
import models.cvae as vm

In [ ]:
input_shape = (128,128,3)

In [ ]:
reload(nm)
encoder = nm.Encoder(input_shape=input_shape, n_extra_layers=2)
encoder.summary()

In [ ]:
reload(nm)
encoder = nm.Decoder(input_shape=input_shape, n_extra_layers=2)
encoder.summary()

In [ ]:
reload(gm)
discriminator = gm.Discriminator(input_shape)
#discriminator.features.build((None, *input_shape))
#discriminator.features.summary()
discriminator.build((None, *input_shape))
discriminator.summary()

In [ ]:
reload(gm)
model = gm.GANomaly(input_shape)
model.compile()
model.build((None, *input_shape))
model.summary()

In [ ]:
model.metrics

In [ ]:
import tensorflow as tf

#optimizers = dict(filter(lambda kv: 'optimizer' in kv[0], model.__dict__.items()))
#print(optimizers)

optimizers = {k: v for k, v in model.__dict__.items() if isinstance(v, tf.keras.optimizers.Optimizer)}
print(optimizers)

for k, v in {}: #model.__dict__.items():
    if 'optimizer' not in k:
        continue
    #print(k, v)
    #print(getattr(model, k))
    old_lr = float(tf.keras.backend.get_value(v.lr))
    new_lr = old_lr * 0.1
    print(type(old_lr), old_lr, type(new_lr), new_lr)
    #print("adjusting {}.lr from {} to {}".format(k, old_lr, new_lr))
    tf.keras.backend.set_value(v.lr, new_lr)

In [ ]:
reload(vm)
cvae = vm.CVAE(input_shape=input_shape, intermediate_size=100, latent_size=100)
cvae.build((None, *input_shape))
cvae.summary()

In [ ]:
reload(vm)
cvae = vm.CVAE(input_shape=input_shape, intermediate_size=0, latent_size=100)
cvae.build((None, *input_shape))
cvae.summary()

In [ ]:
import tensorflow as tf

In [ ]:
encoder = tf.keras.Sequential(
    tf.keras.layers.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='same'
    ),
)
encoder.build((None, 16, 16, 16))
encoder.summary()

In [ ]:
decoder = tf.keras.Sequential(
    tf.keras.layers.Conv2DTranspose(
        filters=16,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding='valid'
    ),
)
decoder.build((None, 16, 16, 16))
decoder.summary()

# Experiments with transposed convolution

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
X = np.array([
        [0, 1],
        [2, 3]
    ], dtype=np.float32)
K = np.array([
        [0, 1],
        [2, 3]
    ], dtype=np.float32)

## Own transpose layer with numpy

In [ ]:
def conv2d_transpose(X, K, strides=(1, 1), padding='valid'):
    Y = np.zeros((
        (X.shape[0] - 1) * strides[0] + K.shape[0],
        (X.shape[1] - 1) * strides[1] + K.shape[1]
    ))
    for i in range(X.shape[0]):
        strided_i = i * strides[0]
        for j in range(X.shape[1]):
            strided_j = j * strides[1]
            Y[strided_i:strided_i + K.shape[0], strided_j:strided_j + K.shape[1]] += X[i, j] * K
    if padding.lower() == 'valid':
        return Y
    else:
        p_left   = (K.shape[0] - strides[0]) // 2
        p_right  = K.shape[0] - strides[0] - p_left
        p_top    = (K.shape[1] - strides[1]) // 2
        p_bottom = K.shape[1] - strides[1] - p_top
        return Y[p_left:Y.shape[0] - p_right, p_top:Y.shape[1] - p_bottom]

In [ ]:
conv2d_transpose(X, K, padding='valid')

In [ ]:
conv2d_transpose(X, K, padding='same')

In [ ]:
conv2d_transpose(X, K, strides=(2, 2), padding='valid')

In [ ]:
conv2d_transpose(X, K, strides=(2, 2), padding='same')

## Compare results with tf.nn.conv2d_transpose

In [ ]:
Y = tf.nn.conv2d_transpose(
    X.reshape((1, 2, 2, 1)), # [batch, height, width, in_channels]
    filters=K.reshape((2, 2, 1, 1)), # [height, width, output_channels, in_channels]
    output_shape=(1, 3, 3, 1), # [batch, height, width, output_channels]
    strides=(1, 1),
    padding='VALID',
)
tf.squeeze(Y)

In [ ]:
Y = tf.nn.conv2d_transpose(
    X.reshape((1, 2, 2, 1)), # [batch, height, width, in_channels]
    filters=K.reshape((2, 2, 1, 1)), # [height, width, output_channels, in_channels]
    output_shape=(1, 2, 2, 1), # [batch, height, width, output_channels]
    strides=(1, 1),
    padding='SAME',
)
tf.squeeze(Y)

In [ ]:
Y = tf.nn.conv2d_transpose(
    X.reshape((1, 2, 2, 1)), # [batch, height, width, in_channels]
    filters=K.reshape((2, 2, 1, 1)), # [height, width, output_channels, in_channels]
    output_shape=(1, 4, 4, 1), # [batch, height, width, output_channels]
    strides=(2, 2),
    padding='VALID',
)
tf.squeeze(Y)

In [ ]:
Y = tf.nn.conv2d_transpose(
    X.reshape((1, 2, 2, 1)), # [batch, height, width, in_channels]
    filters=K.reshape((2, 2, 1, 1)), # [height, width, output_channels, in_channels]
    output_shape=(1, 4, 4, 1), # [batch, height, width, output_channels]
    strides=(2, 2),
    padding='SAME',
)
tf.squeeze(Y)

## Compare results with tf.keras.layers.Conv2DTranspose

In [ ]:
layer = tf.keras.layers.Conv2DTranspose(
    filters=1,
    kernel_size=K.shape,
    strides=(1, 1),
    padding='VALID',
    use_bias=False,
    kernel_initializer='zeros',
    bias_initializer='zeros'
)
layer.build((None, 2, 2, 1))
layer.set_weights([K.reshape((2, 2, 1, 1))]) # [height, width, output_channels, in_channels]

Y = layer(X.reshape((1, 2, 2, 1)))  # [batch, height, width, in_channels]
tf.squeeze(Y)

In [ ]:
layer = tf.keras.layers.Conv2DTranspose(
    filters=1,
    kernel_size=K.shape,
    strides=(1, 1),
    padding='SAME',
    use_bias=False,
    kernel_initializer='zeros',
    bias_initializer='zeros'
)
layer.build((None, 2, 2, 1))
layer.set_weights([K.reshape((2, 2, 1, 1))]) # [height, width, output_channels, in_channels]

Y = layer(X.reshape((1, 2, 2, 1)))  # [batch, height, width, in_channels]
tf.squeeze(Y)

In [ ]:
layer = tf.keras.layers.Conv2DTranspose(
    filters=1,
    kernel_size=K.shape,
    strides=(2, 2),
    padding='VALID',
    use_bias=False,
    kernel_initializer='zeros',
    bias_initializer='zeros'
)
layer.build((None, 2, 2, 1))
layer.set_weights([K.reshape((2, 2, 1, 1))]) # [height, width, output_channels, in_channels]

Y = layer(X.reshape((1, 2, 2, 1)))  # [batch, height, width, in_channels]
tf.squeeze(Y)

In [ ]:
layer = tf.keras.layers.Conv2DTranspose(
    filters=1,
    kernel_size=K.shape,
    strides=(2, 2),
    padding='SAME',
    use_bias=False,
    kernel_initializer='zeros',
    bias_initializer='zeros'
)
layer.build((None, 2, 2, 1))
layer.set_weights([K.reshape((2, 2, 1, 1))]) # [height, width, output_channels, in_channels]

Y = layer(X.reshape((1, 2, 2, 1)))  # [batch, height, width, in_channels]
tf.squeeze(Y)